# Constitution de l'échantillon de données étiquetées

L'objet de ce notebook est de produire un échantillon données du PIM, avec les fiches techniques associées.
Elles seront ensuite associées manuellement à la liste d'ingrédients qu'elles contiennent.

## Récupération des données

### Préambule technique

In [1]:
# setting up sys.path for relative imports
from pathlib import Path
import sys
project_root = str(Path(sys.path[0]).parents[1].absolute())
if project_root not in sys.path:
    sys.path.append(project_root)

In [43]:
# imports and customization of diplay
import os
import pandas as pd
pd.options.display.min_rows = 6
pd.options.display.width=108
from sklearn.model_selection import train_test_split

from src.pimapi import Requester

### Récupération des données, et de la présences de fiches techniques

Pour constituer l'échantillon, on va d'abord extraire quelques informations du PIM, et particulièrement le type de produit.
On récupèrera aussi le fait que les produits ont ou non une fiche technique fournisseur associée.

In [7]:
requester = Requester('prd')
# Let's fetch the full content of PIM system
requester.fetch_all_from_PIM()
requester.result

Done


[<Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>]

In [8]:
mapping = {'uid': 'uid',
           'designation': 'title',
           'state': 'state',
           'ingredients': 'properties.pprodc:ingredientsList',
           'type': 'properties.pprodtop:typeOfProduct'}
df = requester.file_report_from_result(mapping=mapping, index='uid') # , record_path='entries') 
df

,designation,state,ingredients,type,has_supplierdatasheet,has_supplierlabel
uid,,,,,,
afee12c7-177e-4a68-9539-8cbb68442503,DESTR D'ODEURS AIR&TEXTILES 750CCX6 DESODOR U2,product.waiting.supplier.validation,None,chemistry,False,False
7d390121-17e8-43bf-a357-9d06b79d2d47,THÉ VERT AGRUME BTE 25S FRAICH LIPTON,product.waiting.supplier.validation,None,grocery,False,False
f234cd84-c8f6-433f-85ec-6e0b6980adc6,T WHEAT 30 A 18X6 52C MISSION 1620,product.waiting.supplier.validation,"WHEAT flour (55%), water, vegetable fat (palm)...",grocery,True,True
...,...,...,...,...,...,...
ef42a938-2203-446e-8d28-9fd27c6d3146,3D VENT FRAIS 5LX4 DESODOR U2,product.waiting.supplier.validation,None,chemistry,False,False
68f5d81b-7f91-40a0-8504-0ec320a86de4,NETTOYANT INOX 500ML LOT 2X6 KING,product.waiting.supplier.validation,None,chemistry,False,False
6dfce29e-fd4c-4670-9f9c-5c02a5b4d52a,DESINFECTANT 3D+ 750CCX6 DESODOR U2,product.waiting.supplier.validation,None,chemistry,False,False


## Constitution de l'échantillon

On va constituer l'échantillon en appliquant les règles suivantes : 
- on construit un échantillon de 500 produits
- on conserve les produits de type Epicerie et Boisson non alcoolisée
- on conserve les produits qui possèdent une fiche technique fournisseur
- on fait un échantillon stratifié par type de produit (Epicerie / Boisson)

In [12]:
filtered_df = df.loc[(df.type.isin(['grocery', 'nonAlcoholicDrink']))
                     & (df.has_supplierdatasheet)]
train, ground_truth_df = train_test_split(filtered_df,
                                          test_size=500,
                                          random_state=42,
                                          stratify=filtered_df.type)
ground_truth_df

,designation,state,ingredients,type,has_supplierdatasheet,has_supplierlabel
uid,,,,,,
49428283-104a-4092-966f-07b974112836,Jus de pomme Granny Smith en bouteille verre 1...,product.validate,100 % pur jus de Pommes Granny Smith.,nonAlcoholicDrink,True,False
c679c923-cf39-4e40-b072-7474928450c6,Purée de pomme de terre en sac 5 kg LUTOSA,product.validate,"Pommes de terre déshydratées (99 %), émulsifia...",grocery,True,True
ef8afbbb-efbc-4fdf-aa1c-6f664f2c1073,CÂPRE FINE AU VINAIGRE EN BOÎTE 4/4 VITALY'S,product.validate,"Câpres, eau, vinaigre, sel",grocery,True,False
...,...,...,...,...,...,...
aff93d0f-a94c-4e9b-a6ca-d69115e3d2eb,Thon Listao en morceaux au naturel en boîte 4/...,product.validate,"THON listao, eau, sel",grocery,True,True
a01039be-eb79-454f-a384-c142d1d80d0c,"Tajine végétarienne en barquette 2,3 kg CHRIST",product.validate,"Légumes 48,1% : pommes de terre 18,2%, carotte...",grocery,True,True
97c80844-08c7-45c6-82ac-43ab8a45f2e4,Spécialité pomme-fraise en gourde 90 g ANDROS,product.waiting.pomona.validation.niv3,"Ingrédients : Pommes 74%, fraises 20%, sucre, ...",grocery,True,True


**Remarque** : malgré l'utilisation d'un `random_state` fixé, l'échantillon généré n'est pas toujours le même à chaque exécution. En effet, comme la liste de produits varie au fil du temps (nouveaux référencements, périmètre des filtres qui change), le résultat du `train_test_split` peut varier.


Il s'agit ici seulement d'illustrer la méthode utilisée.


## Export des pièces jointes du PIM et constitution du fichier d'étiquettes

On exporte ensuite le contenu du PIM sur le disque, afin d'avoir les fiches techniques simplement à disposition.

In [14]:
requester.fetch_list_from_PIM(ground_truth_df.index, batch_size=20)
requester.dump_data_from_result(update_directory=False, root_path=os.path.join('.', 'ground_truth_to_del'))
requester.dump_files_from_result(update_directory=False, root_path=os.path.join('.', 'ground_truth_to_del'))

Done
Done
Launching 25 threads.
Thread complete!
Thread complete!
Thread complete!
Thread complete!
Thread complete!
Thread complete!
Thread complete!
Thread complete!
Thread complete!
Thread complete!
Thread complete!
Thread complete!
Thread complete!
Thread complete!
Thread complete!
Thread complete!
Thread complete!
Thread complete!
Thread complete!
Thread complete!
Thread complete!
Thread complete!
Thread complete!
Thread complete!
Thread complete!
Done


On exporte également au format csv les uids des produits et les libellés associés (pour s'assurer qu'il n'y a pas eu de confusion lorsqu'on lit une fiche technique).

In [15]:
ground_truth_df['designation'].to_csv(os.path.join('.', 'ground_truth_to_del', 'uid.csv'),
                                      header=True,
                                      encoding='utf-8-sig')

On teste également la possibilité de recharger les données depuis le fichier csv, une fois qu'il a été renseigné à la main dans excel.

In [20]:
pd.read_csv(os.path.join('..', '..', 'ground_truth', 'manually_labelled_ground_truth.csv'),
            sep=';',
            encoding='latin-1',
            index_col='uid')

,designation,ingredients
uid,,
a0492df6-9c76-4303-8813-65ec5ccbfa70,Concentré liquide Asian en bouteille 980 ml CHEF,"Eau, maltodextrine, sel, arômes, sucre, arôme ..."
d183e914-db2f-4e2f-863a-a3b2d054c0b8,Pain burger curry 80 g CREATIV BURGER,"Farine de blé T65, eau, levure, vinaigre de ci..."
ab48a1ed-7a3d-4686-bb6d-ab4f367cada8,Macaroni en sachet 500 g PANZANI,- 100% Semoule de BLE dur de qualité supérieur...
...,...,...
e67341d8-350f-46f4-9154-4dbbb8035621,PRÉPARATION POUR CRÈME BRÛLÉE BIO 6L,"Sucre roux de canne*° (64%), amidon de maïs*, ..."
a8f6f672-20ac-4ff8-a8f2-3bc4306c8df3,Céréales instantanées en poudre saveur caramel...,"Farine 87,1 % (Blé (GLUTEN), Blé hydrolysé (GL..."
0faad739-ea8c-4f03-b62e-51ee592a0546,"FARINE DE BLÉ TYPE 45, 10KG",Farine de blé T45


## Résultat de l'étiquetage manuel

Le résultat de l'étiquetage manuel est le suivant :

In [3]:
df_gt = pd.read_csv(os.path.join('..', '..', 'ground_truth', 'manually_labelled_ground_truth.csv'),
                    sep=';',
                    encoding='latin-1',
                    index_col='uid')

def to_latex_newline(text):
    return(text.replace('\n', '  '))

with pd.option_context("max_colwidth", 1000):
    print(df_gt)
    df_gt.to_latex(Path('..') / 'tbls' / 'ground_truth.tex',
                   index=False,
                   index_names=False,
                   column_format='p{5cm}p{10cm}',
                   formatters=[to_latex_newline, to_latex_newline],
                   longtable=True,
                   na_rep='-',
                   escape=True,
                   )

                                                                                                designation  \
uid                                                                                                           
a0492df6-9c76-4303-8813-65ec5ccbfa70                       Concentré liquide Asian en bouteille 980 ml CHEF   
d183e914-db2f-4e2f-863a-a3b2d054c0b8                                  Pain burger curry 80 g CREATIV BURGER   
ab48a1ed-7a3d-4686-bb6d-ab4f367cada8                                       Macaroni en sachet 500 g PANZANI   
...                                                                                                     ...   
e67341d8-350f-46f4-9154-4dbbb8035621                                   PRÉPARATION POUR CRÈME BRÛLÉE BIO 6L   
a8f6f672-20ac-4ff8-a8f2-3bc4306c8df3  Céréales instantanées en poudre saveur caramel en boîte 400 g BLEDINA   
0faad739-ea8c-4f03-b62e-51ee592a0546                                            FARINE DE BLÉ TYPE 45, 10KG   



## Comparatif entre les données étiquetées et le contenu du PIM

On peut comparer le contenu des listes d'ingrédients du PIM et les données étiquetées.


In [5]:
requester = Requester('prd')
requester.fetch_all_from_PIM()
requester.result

Done


On récupère le contenu du PIM 

In [7]:
df = requester.result_to_dataframe()
pim_ds = df['properties.pprodc:ingredientsList']
pim_ds

,entity-type,repository,path,type,state,parentRef,isCheckedOut,isVersion,isProxy,changeToken,...,properties.pprodqmdd:manufacturingDiagram.length,properties.pprodqmdd:manufacturingDiagram.data,properties.pprodqmdd:secondaryPackagingPhoto.name,properties.pprodqmdd:secondaryPackagingPhoto.mime-type,properties.pprodqmdd:secondaryPackagingPhoto.encoding,properties.pprodqmdd:secondaryPackagingPhoto.digestAlgorithm,properties.pprodqmdd:secondaryPackagingPhoto.digest,properties.pprodqmdd:secondaryPackagingPhoto.length,properties.pprodqmdd:secondaryPackagingPhoto.data,properties.notif:notifications
uid,,,,,,,,,,,,,,,,,,,,,
afee12c7-177e-4a68-9539-8cbb68442503,document,default,/default-domain/pomSupplierWorkspace/SICO/DEST...,pomProduct,product.waiting.supplier.validation,a58845c0-cab3-492f-b48d-531f146c3777,True,False,False,17-0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7d390121-17e8-43bf-a357-9d06b79d2d47,document,default,/default-domain/pomSupplierWorkspace/UNILEVER_...,pomProduct,product.waiting.supplier.validation,a37abc27-f485-4ae9-921b-f761f16c8c1c,False,False,False,15-0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
f234cd84-c8f6-433f-85ec-6e0b6980adc6,document,default,/default-domain/pomSupplierWorkspace/AZTECA_FO...,pomProduct,product.waiting.supplier.validation,3ff7819a-a392-493f-beb8-0b323ac331c7,True,False,False,33-0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ef42a938-2203-446e-8d28-9fd27c6d3146,document,default,/default-domain/pomSupplierWorkspace/SICO/DETE...,pomProduct,product.waiting.supplier.validation,a58845c0-cab3-492f-b48d-531f146c3777,True,False,False,17-0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68f5d81b-7f91-40a0-8504-0ec320a86de4,document,default,/default-domain/pomSupplierWorkspace/SICO/NETT...,pomProduct,product.waiting.supplier.validation,a58845c0-cab3-492f-b48d-531f146c3777,True,False,False,17-0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6dfce29e-fd4c-4670-9f9c-5c02a5b4d52a,document,default,/default-domain/pomSupplierWorkspace/SICO/SPRA...,pomProduct,product.waiting.supplier.validation,a58845c0-cab3-492f-b48d-531f146c3777,True,False,False,17-0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


On charge le csv des données étiquetées : 

In [61]:
df_gt = pd.read_csv(os.path.join('..', '..', 'ground_truth', 'manually_labelled_ground_truth.csv'),
                    sep=';',
                    encoding='latin-1',
                    index_col='uid')
df_gt

,designation,ingredients
uid,,
a0492df6-9c76-4303-8813-65ec5ccbfa70,Concentré liquide Asian en bouteille 980 ml CHEF,"Eau, maltodextrine, sel, arômes, sucre, arôme ..."
d183e914-db2f-4e2f-863a-a3b2d054c0b8,Pain burger curry 80 g CREATIV BURGER,"Farine de blé T65, eau, levure, vinaigre de ci..."
ab48a1ed-7a3d-4686-bb6d-ab4f367cada8,Macaroni en sachet 500 g PANZANI,- 100% Semoule de BLE dur de qualité supérieur...
...,...,...
e67341d8-350f-46f4-9154-4dbbb8035621,PRÉPARATION POUR CRÈME BRÛLÉE BIO 6L,"Sucre roux de canne*° (64%), amidon de maïs*, ..."
a8f6f672-20ac-4ff8-a8f2-3bc4306c8df3,Céréales instantanées en poudre saveur caramel...,"Farine 87,1 % (Blé (GLUTEN), Blé hydrolysé (GL..."
0faad739-ea8c-4f03-b62e-51ee592a0546,"FARINE DE BLÉ TYPE 45, 10KG",Farine de blé T45


Comme l'index de la series des données issues du PIM, et du dataframe de la ground truth est le même (l'uid du produit), on peut faire très simplement la jointure via la méthode `join` :

In [62]:
merged = (df_gt.join(pim_ds)
               .rename({'ingredients': 'Ingrédients de la ground truth', 
                        'properties.pprodc:ingredientsList': 'Ingrédients du PIM'},
                       axis=1)
         )
merged

,designation,Ingrédients de la ground truth,Ingrédients du PIM
uid,,,
a0492df6-9c76-4303-8813-65ec5ccbfa70,Concentré liquide Asian en bouteille 980 ml CHEF,"Eau, maltodextrine, sel, arômes, sucre, arôme ...","Eau, maltodextrine, sel, arômes, sucre, arôme ..."
d183e914-db2f-4e2f-863a-a3b2d054c0b8,Pain burger curry 80 g CREATIV BURGER,"Farine de blé T65, eau, levure, vinaigre de ci...","Farine de BLE T65, eau, levure, huile de colz..."
ab48a1ed-7a3d-4686-bb6d-ab4f367cada8,Macaroni en sachet 500 g PANZANI,- 100% Semoule de BLE dur de qualité supérieur...,100% Semoule de BLE dur de qualité supérieure
...,...,...,...
e67341d8-350f-46f4-9154-4dbbb8035621,PRÉPARATION POUR CRÈME BRÛLÉE BIO 6L,"Sucre roux de canne*° (64%), amidon de maïs*, ...","Sucre roux de canne*°(64%), amidon de maïs*, p..."
a8f6f672-20ac-4ff8-a8f2-3bc4306c8df3,Céréales instantanées en poudre saveur caramel...,"Farine 87,1 % (Blé (GLUTEN), Blé hydrolysé (GL...","Farine 87,1 % (BLE (GLUTEN), BLE hydrolysé (GL..."
0faad739-ea8c-4f03-b62e-51ee592a0546,"FARINE DE BLÉ TYPE 45, 10KG",Farine de blé T45,farine de BLE T45


On peut compter les égalités strictes entre les ingrédients du PIM et ceux de la ground truth :

In [63]:
merged['equals'] = (merged['Ingrédients du PIM'] == merged['Ingrédients de la ground truth'])
merged['equals'].value_counts()

False    450
True      50
Name: equals, dtype: int64

Seules 50 listes d'ingrédients sont strictement identiques. Si on compare les listes qui ne le sont pas, on obtient : 

In [64]:
diff = merged.loc[~merged['equals'], ['Ingrédients du PIM', 'Ingrédients de la ground truth']]
for i in range(20):
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('Issu du PIM :')
    print(diff.iloc[i].loc['Ingrédients du PIM'])
    print('----------------------------------------------------')
    print('Issu de la ground truth :')
    print(diff.iloc[i].loc['Ingrédients de la ground truth'])
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++\n')

++++++++++++++++++++++++++++++++++++++++++++++++++++
Issu du PIM :
Farine de BLE T65, eau, levure,  huile de colza, sel, vinaigre de cidre, assaisonnement poudre de curry, agent de traitement de la farine : acide ascorbique, émulsifiant : E471
----------------------------------------------------
Issu de la ground truth :
Farine de blé T65, eau, levure, vinaigre de cidre, huile de colza, assaisonnement poudre de curry, sel, acide ascorbique, émulsifiant : E471.
++++++++++++++++++++++++++++++++++++++++++++++++++++

++++++++++++++++++++++++++++++++++++++++++++++++++++
Issu du PIM :
100% Semoule de BLE dur de qualité supérieure
----------------------------------------------------
Issu de la ground truth :
- 100% Semoule de BLE dur de qualité supérieure
- Contient du gluten
Si le numéro de lot contient la lettre N : peu contenir de l'oeuf
++++++++++++++++++++++++++++++++++++++++++++++++++++

++++++++++++++++++++++++++++++++++++++++++++++++++++
Issu du PIM :
Fève de tonka, taux de coumarine 

On peut sortir un tableau des données en écart, de manière basique :

In [92]:
with pd.option_context("max_colwidth", 100000):
    tex_str = (
    diff.sample(10, random_state=42)
        .to_latex(index=False,
                  index_names=False,
                  column_format='p{7cm}p{7cm}',
                  na_rep='-',
                 )
        .replace(r'\textbackslash n', '\\newline ')
    )

with open(Path('..') / 'tbls' / 'ingredient_comparison.tex', 'w') as file:
    file.write(tex_str)